# Preparação dos dados do Microdados do ENADE para a classificação

## Alunos:
    - Berly Joaquin
    - Leonardo Lopes

## Objetivos

O objetivo deste script é preparar os dados para serem utilizados em classificadores.

## Bibliotecas Utilizadas

Para a análise feita neste trabalho, utilizamos as bibliotecas **Pandas** e **Numpy** da linguagem Python.
Na classificação, foi utilizada a ferramenta **Weka**, um software de mineração de dados escrito em Java, disponível em: http://www.cs.waikato.ac.nz/ml/weka/

In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import subprocess
# from sklearn.tree import DecisionTreeClassifier, export_graphviz

## Preparando os dados

Na etapa da preparação dos dados, foi necessário tratar as informações obtidas nas bases de dados fornecidas.

### Tipos de Informações

* **Ano**: representa ano em que o exame foi realizado e está disponível na base de dados na coluna `nu_ano` sem maior necessidade de tratamentos de dados;
* **Idade**: análoga à informação de ano, está disponível na base de dados de forma amigável na coluna `nu_idade` e não foi necessário tratar estes dados;
* **Curso**: curso do aluno (sempre será Ciência da Computação)
* **Renda**: representa a renda da família em número de salários mínimos (independente do ano do exame), foi o tipo de dado que apresentou maiores dificuldades no seu tratamento, uma vez que a informação aparece com diferetes nomes de colunas na base de dados original e seus valores categórios não nos trazem grandes informações de forma imediata. Para tal, transformamos o a informação categórica em dados numéricos (a categoria foi mapeada para seu valor médio)
* **Bolsa**: tipo de bolsa do aluno
    * a) Nenhum, pois meu curso é gratuito
    * b) Nenhum, embora meu curso não seja gratuito
    * c) ProUni integral
    * d) ProUni parcial, apenas
    * e) FIES, apenas
    * f) ProUni Parcial e FIES
    * g) Bolsa oferecida por governo estadual, distrital ou municipal
    * h) Bolsa oferecida pela própria instituição
    * i) Bolsa oferecida por outra entidade (empresa, ONG, outra)
    * j) Financiamento oferecido pela própria instituição
    * k) Financiamento bancário

In [2]:
dict_select_bolsa_13_14={ 'a': True,
                          'b': True,
                          'c': True,
                          'd': True,
                          'e': True,
                          'g': True,
                          'h': True,
                          'i': True,
                          'j': True,
                          'k': True,
}


In [3]:
def clean_data(dataframe, renda, bolsa, dict_renda=None):
    new_dataframe = pd.DataFrame(columns=['ano', 'idade'])
    new_dataframe['ano'] = dataframe['nu_ano']
    new_dataframe['idade'] = dataframe['nu_idade']
    new_dataframe['curso'] = dataframe['co_grupo']
    new_dataframe['bolsa'] = dataframe[bolsa].str.lower()
    
    if dict_renda is None:
        dict_renda = { 'a': 1.5, 'b': 2.25, 'c': 3.75, 'd': 5.25, 'e': 8.0, 'f': 15.0, 'g': 30.0 }
    
    r = dataframe[renda].str.lower()
    
    new_dataframe['renda'] = r.map(dict_renda)
    def m(item):
        return round(item/20)
    new_dataframe['nota'] = dataframe['nt_ger'].map(m)

    def f(i):
        return dict_select_bolsa_13_14.get(new_dataframe["bolsa"][i], False)

    return new_dataframe.select(f)

### Obtendo os dados

Os dados da base de dados foram obtidos a partir do arquivo .csv de cada ano

In [4]:
def is_number(s):
    try:
        f = float(s)
        if f >= 0:
            return True
        return False
    except ValueError:
        return False


In [5]:
def select_clause(df):
    def f(i):
        if df["co_grupo"][i] == 4004 and is_number(df["nt_ger"][i]):
            return True
        else:
            return False
    return f

In [6]:
# dict_renda_2011 = { 'b': 1.5, 'c': 2.25, 'd': 3.75, 'e': 5.25, 'f': 8.0, 'g': 15.0, 'h': 30.0 }

# df10 = clean_data(pd.read_csv('microdados_enade_2010.csv', encoding='latin1', delimiter=';'), 'QE_I5', 'QE_I10')
# df11 = clean_data(pd.read_csv('microdados_enade_2011.csv', encoding='latin1', delimiter=';'), 'co_rs_s5','co_rs_s10', dict_renda_2011)
# df12 = clean_data(pd.read_csv('microdados_enade_2012.csv' ,encoding='latin1', delimiter=';'), 'qe_i5', 'qe_i10')
# df13 = clean_data(pd.read_csv('microdados_enade_2013.csv', delimiter=';'), 'co_rs_s7', 'co_rs_s10' )
data = pd.read_csv('microdados_enade_2014.csv', delimiter=';', low_memory=False)
df14 = clean_data(data.select(select_clause(data)),  'qe_i8', 'qe_i11')
df = df14

# dfs = [df10, df11, df12, df13, df14]

In [7]:
# df = pd.DataFrame()

# df = df.append(dfs)
pd.options.display.float_format = '{:.2f}'.format

df.describe()

,ano,idade,curso,renda,nota
count,8174.00,8174.00,8174.00,8174.00,8174.00
mean,2014.00,25.28,4004.00,6.55,2.23
std,0.00,4.39,0.00,5.25,0.74
min,2014.00,18.00,4004.00,1.50,0.00
25%,2014.00,22.00,4004.00,2.25,2.00
50%,2014.00,24.00,4004.00,5.25,2.00
75%,2014.00,27.00,4004.00,8.00,3.00
max,2014.00,65.00,4004.00,30.00,5.00


In [8]:
df.to_csv(path_or_buf='output.csv')

In [9]:
# dict_bolsa_13_14 Esta variavel servirá como dicionario para os anos 2013 e 2014

dict_bolsa_13_14={'a': 'Nenhum, pois meu curso é gratuito',
                  'b':'Nenhum, embora meu curso não seja gratuito',
                  'c':'ProUni integral', 
                  'd':'ProUni parcial, apenas',
                  'e':'FIES, apenas', 'f': 'ProUni Parcial e FIES',
                  'g':'Bolsa oferecida por governo estadual, distrital ou municipal', 
                  'h' :'Bolsa oferecida pela própria instituição', 
                  'i':'Bolsa oferecida por outra entidade (empresa, ONG, outra)', 
                  'j' : 'Financiamento oferecido pela própria instituição',
                  'k': 'Financiamento bancário'}
